In [6]:
import numpy as np
from scipy.stats import bernoulli, norm, poisson
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
import torch
import pyro

from pgmpy.factors.discrete import DiscreteFactor
from pyro.distributions import Bernoulli, Categorical, Gamma, Normal, Poisson

In [8]:
print(Categorical(probs=torch.tensor([.45, .30, .25])))

Categorical(probs: torch.Size([3]))


In [17]:
Categorical(probs=torch.tensor([.45, .30, .25])).sample()

tensor(1)

### Sampling from Distributions

In [22]:
z = Gamma(7.5, 1.0).sample()
x = Poisson(z).sample()
y = Bernoulli(x/(5+x)).sample()

x,y,z

(tensor(9.), tensor(1.), tensor(7.3949))

<center>$z \sim p(Z)$<center>

<center>$x \sim P(Y|Z=z)$<center>

<center>$y_i \sim P(Y|X=x)$<center>
    
<center>$y = \sum_{i}^x y_i$<center>

In [29]:
z = Gamma(7.5, 1.0).sample()
x = Poisson(z).sample()
y = torch.tensor(0.0)
for i in range(int(x)):
    print(i)
    y += Bernoulli(.5).sample()

x,y,z

0
1
2
3
4
5


(tensor(6.), tensor(0.), tensor(8.0881))

### Random process (X,Y,Z)

In [31]:
def random_process():
    z = pyro.sample('z', Gamma(7.5, 1.0))
    x = pyro.sample('x', Poisson(z))
    y = torch.tensor(0.0)
    for i in range(int(x)):
        y += pyro.sample(f'y{i}', Bernoulli(.5))
        
    return y

In [32]:
samples = torch.stack([random_process() for _ in range(100)])
samples

tensor([ 3.,  3.,  4.,  0.,  3.,  4.,  2.,  3.,  2.,  5.,  2.,  2.,  2.,  1.,
         2.,  1.,  8.,  7.,  2.,  8.,  0.,  0.,  3.,  1.,  3.,  1.,  4.,  6.,
         3., 11.,  4.,  3., 10.,  1.,  2.,  8.,  0.,  1.,  4.,  1.,  3.,  2.,
         3.,  2.,  7.,  6.,  5.,  2.,  1.,  3., 10.,  1.,  8.,  5.,  3.,  4.,
         3.,  3.,  5.,  4.,  3.,  3.,  5.,  3.,  1.,  2.,  3.,  3.,  4.,  1.,
         1.,  7.,  4.,  4.,  5.,  2.,  3.,  2.,  0.,  7.,  3.,  0.,  2.,  6.,
         2.,  5.,  3.,  6.,  2.,  2.,  0.,  5.,  8.,  3.,  5.,  3.,  3.,  1.,
         6.,  1.])

### Expectation of Y $E[Y]$

In [33]:
samples.mean()

tensor(3.3900)

 ### Expectation of a funtion $E[f(Y)] : f(y) = y^2$

In [34]:
torch.square(samples).mean()

tensor(17.1300)